# 데이콘 코드 유사성 판단 AI 경진대회

| 이름 | 비고 | 
|-------|------|
| 홍영복 | 팀장 |
| 구창회 | 팀원 |
| 이민규 | 팀원 |
| 이재성 | 팀원 |

# open/code 전처리

In [1]:
import os              # 운영체제 os
import pandas as pd    # Pandas
import numpy as np     # numpy 
import glob            # 파일을 읽어서 반환 glob 
from tqdm import tqdm  # 코드 진행 상황 확인
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

CODE_FOLDER = "./open/code/"

In [2]:
# open/code에 있는 모들 .py 파일을 읽어 데이터프레임으로 반환
def make_df(path_name) :
    problem_folders = os.listdir(path_name)                                 # 해당 경로의 모든 파일을 읽어 리스트로 반환
    code_list = []                                                          # .py 파일이 저장될 code_list
    problem_num = []                                                        # .py 파일의 problem 번호를 저장할 problem_num
    problem_py_num = []
    except_file = []                                                        # 문자열로 바뀐 .py 파일의 길이가 너무 길면 제외, 어떤 파일을 제외했는지 저장할 except_file
    
    for problem_folder in tqdm(problem_folders) :                
        for py_file in glob.glob( path_name + problem_folder + "/*.py") :   # glob.glob 을 통해 .py 확장자가 붙은 파일만을 읽어옴.
            py_code = []                                                    # 하나의 .py 파일의 내용을 담을 py_code, 파일을 새로 열때마다 초기화. 
            long_annotation = 0                                             # 긴줄 주석 여부를 확인하는 long_annotation
            long_str = 0
                        
            with open(py_file, 'r', encoding='utf-8') as file :
                lines = file.readlines()                                    # readlines() 를 통해 파일의 모든 내용을 문자열로 반환
                
                for line in lines :                                         # 그 중 리스트 요소 하나에 해당하는 라인 한줄에 대해서 진행
                    line = line.rstrip().lstrip()                           #  문자열 앞 뒤로 공백 제거
                    
                    # 만약 한 줄 주석으로 시작하면 새로 반복
                    if line.startswith('#') :
                        continue
                    elif line.count('"""') == 0 and line.startswith('\"') and line[1:].endswith('\"') : # "문자열"
                        continue  
                    elif line.count("'''") == 0 and line.startswith("\'") and line[1:].endswith("\'") :  # '문자열'
                        continue
                    
                    # """를 찾았을 때
                    if line.count('"""') >= 1 : 
                        if line.count('"""') % 2 == 0 and line.startswith('"""') and line[3:].endswith('"""') :
                            continue
                        elif line.count("'''") % 2 == 0 and line.startswith("'''") and line[3:].endswith("'''") :
                            continue  
    
                    if line.count('"""') % 2 != 0 and long_annotation == 0 and long_str == 0 : 
                        if line.startswith('"""') :         
                            long_annotation = 1                          
                            continue
                        else : 
                            long_str = 1
                    elif line.count('"""') % 2 != 0 and long_annotation == 1 and long_str == 0 :
                        long_annotation = 0
                        continue    
                    elif line.count('"""') % 2 != 0 and long_annotation == 0 and long_str == 1 :
                        long_str = 0
                    
                    # '''를 찾았을 때
                    if line.count("'''") >= 1 : 
                        if line.count('"""') % 2 == 0 and line.startswith('"""') and line[3:].endswith('"""') :
                            continue
                        elif line.count("'''") % 2 == 0 and line.startswith("'''") and line[3:].endswith("'''") :
                            continue  
    
                    if line.count("'''") % 2 != 0 and long_annotation == 0 and long_str == 0 : 
                        if line.startswith("'''") :         
                            long_annotation = 1                          
                            continue
                        else : 
                            long_str = 1
                    elif line.count("'''") % 2 != 0 and long_annotation == 1 and long_str == 0 :
                        long_annotation = 0
                        continue    
                    elif line.count("'''") % 2 != 0 and long_annotation == 0 and long_str == 1 :
                        long_str = 0
                    
                    
                    if long_annotation == 1 :                              # 위에 조건문을 통해서 긴 줄 주석이 없는데 long_annotation이 1이면 
                        continue                                           # 다음 라인 탐색
                    
                    if '#' in line :                                        # 문자열 내에 '#'이 있을 경우
                        if line[:line.index('#')].count('\'') % 2 == 0 and line[line.index('#'):].count('\'') % 2 == 0 :
                            line = line[:line.index('#')]                   # '#'을 기준으로 앞,뒤 각각에서 문자열사용이 끝났을 경우 
                                                                            # '#'은 코드 뒤에서 선언된 한줄 주석이므로 '#' 전까지만 사용
                        elif line[:line.index('#')].count('\"') % 2 == 0 and line[line.index('#'):].count('\"') % 2 == 0 :
                            line = line[:line.index('#')]                   # 아래는 큰 따옴표에 대해서도 똑같이 조건문 진행.
                        
                    line = line.replace('\n','')                            # 조건문들을 지나왔으면 줄바꿈 이스케이프 문자는 공백으로
                    line = line.replace('    ','\t')                        # 4칸 공백은 수평탭 이스케이프 문자로 변환
                                                                            # 이때 2칸 공백에 대해 사용하지 않은 것은 4칸과는 달리 코드 곳곳 어디에서나 사용될 수 있기 때문이다.
                    if line == '':                                          # 치환 후 해당 라인이 공백이면
                        continue                                            # 다음 라인 탐색
                        
                    py_code.append(line)                                    # 위의 조건문들에 해당하지 않으면 py_code에 추가.
            
            temp_str = '\n'.join(py_code)                                   # 파일 하나에 대해서 전처리가 끝났으면 줄바꿈 이스케이프 문자로 join연산을 통해 문자열로 반환.
            if temp_str == '' :                                             # 전처리 후의 파일 하나가 공백이라면 
                continue                                                    # 다음 파일을 탐색
            elif len(temp_str) >= 6000 :                                   # 엑셀의 셀 하나의 최대 글자는  32,767이지만 그냥 6,000으로 했다.
                except_file.append(py_file)                                 # 아무튼 코드 파일 하나가 너무 길면 except_file에 추가하고 다음 파일을 탐색
                continue
            else :
                code_list.append(temp_str)                                  # 그 외는 code_list에 파일 하나를 뜻하는 문자열 추가
                problem_num.append(int(problem_folder[7:]))                 # 그 파일이 어떤 problem 폴더에 있는지 나타내는 problem_num에도 추가한다.
                problem_py_num.append(py_file[12:])
            
    print(except_file)                                                      # 3중 for문을 통해 모든 파일을 탐색하고 나서 제외된 파일 출력.
   
    df = pd.DataFrame({'code' : code_list, 'problem_num' : problem_num, 'py_file' : problem_py_num})     # 코드 내용을 나타내는 code_list와 그 코드가 갖는 결과를 나타내는 problem_num을 데이터 프레임 생성
    df.drop_duplicates(['code'], inplace = True, ignore_index = True)                             # 한 코드가 다른 하나의 완전히 똑같은 코드가 30개 있다. 중복 제
    return df

In [3]:
dataset = make_df(CODE_FOLDER)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [04:29<00:00,  1.11it/s]

['./open/code/problem094\\problem094_117.py', './open/code/problem094\\problem094_70.py', './open/code/problem117\\problem117_67.py', './open/code/problem295\\problem295_121.py']


```
problem037\problem037_63.py    4,603   - 답도 없는 이상한 문자열 있음
problem073\problem073_40.py      1,069    - 변수 중 숫자를 쓰는게 있는데 10만,100만 단위임
problem074\problem074_7.py      1,400    - 테스트 한답시고 (1) 변수 사용 
problem085\problem085_101.py      1,912    - 테스트 한답시고 (1) 변수 사용 
problem093\problem093_64.py    1,786    - 테스트 한답시고 (1) 변수 사용 
problem093\problem093_67.py    3,152    - 테스트 한답시고 (1) 변수 사용 
problem095\problem095_50.py    1,192    - 답도 없는 이상한 문자열 있음
problem174\problem174_31.py    1,723    - 변수 중 숫자를 쓰는게 있는데 10만,100만 단위임
problem174\problem174_81.py    1,467    - 변수 중 숫자를 쓰는게 있는데 10만,100만 단위임


(1) 이런 변수가 5~ 8 개 있음.
T1 = """
3
3 4 5
"""
TEST_T1 = """
>>> as_input(T1)
>>> main()
pairwise coprime
"""

```

In [4]:
temp_index = []
li = ['problem037\problem037_63.py',
      'problem073\problem073_40.py',
      'problem074\problem074_7.py',
      'problem085\problem085_101.py',
      'problem093\problem093_64.py',
      'problem093\problem093_67.py',
      'problem095\problem095_50.py',
      'problem174\problem174_31.py',
      'problem174\problem174_81.py']

In [5]:
for i in li :
    temp_index.append(dataset[dataset['py_file'] == i].index[0])
    
temp_index

[5525, 10912, 11094, 12627, 13938, 13941, 14221, 26067, 26122]

In [6]:
dataset.drop(index = temp_index, inplace = True)
dataset = pd.concat([dataset] , ignore_index = True)

In [7]:
dataset

,code,problem_num,py_file
0,"M = 9\nN = 9\ndef main():\nfor i in range(1,M+...",1,problem001\problem001_1.py
1,"list1 = range(1,10)\nlist2 = list1\nfor i in l...",1,problem001\problem001_10.py
2,"for x in range(1, 10):\nfor y in range(1, 10):...",1,problem001\problem001_100.py
3,"for i in range(1,10):\nfor n in range(1, 10):\...",1,problem001\problem001_101.py
4,"for i in range(1,10):\nfor k in range(1,10):\n...",1,problem001\problem001_102.py
...,...,...,...
45051,def prime_factorize(n):\na = [1]\nwhile n % 2 ...,300,problem300\problem300_95.py
45052,"import math\nA,B = map(int,input().split())\ng...",300,problem300\problem300_96.py
45053,def prime_factorization(n):\ntable = []\nfor x...,300,problem300\problem300_97.py
45054,import fractions\nimport math\ndef primefac(n)...,300,problem300\problem300_98.py


In [8]:
tokenizer = Tokenizer()

In [9]:
tokenizer.fit_on_texts(dataset['code'])

In [10]:
sequence = tokenizer.texts_to_sequences(dataset['code'])

In [11]:
ds_list = []

for i in range(len(sequence)) :
    if len(sequence[i]) > 256 :
        ds_list.append(i)
        
    

In [12]:
len(ds_list)

586

In [17]:
dataset.drop(index = ds_list, inplace = True)
dataset = pd.concat([dataset], ignore_index = True)

In [18]:
dataset

,code,problem_num,py_file
0,"M = 9\nN = 9\ndef main():\nfor i in range(1,M+...",1,problem001\problem001_1.py
1,"list1 = range(1,10)\nlist2 = list1\nfor i in l...",1,problem001\problem001_10.py
2,"for x in range(1, 10):\nfor y in range(1, 10):...",1,problem001\problem001_100.py
3,"for i in range(1,10):\nfor n in range(1, 10):\...",1,problem001\problem001_101.py
4,"for i in range(1,10):\nfor k in range(1,10):\n...",1,problem001\problem001_102.py
...,...,...,...
40259,def prime_factorize(n):\na = [1]\nwhile n % 2 ...,300,problem300\problem300_95.py
40260,"import math\nA,B = map(int,input().split())\ng...",300,problem300\problem300_96.py
40261,def prime_factorization(n):\ntable = []\nfor x...,300,problem300\problem300_97.py
40262,import fractions\nimport math\ndef primefac(n)...,300,problem300\problem300_98.py


In [19]:
dataset.to_csv('./open_code_종합.csv')

# train 데이터 생성

In [21]:
dataset = pd.read_csv('./open_code_종합.csv', index_col=0)

In [22]:
train_df = pd.DataFrame(columns = ['code1', 'c1_problem_name', 'code2', 'c2_problem_name', 'similar'])  # 트레인 데이터를 가리키는 빈 데이터 프레임 생성.
train_df

,code1,c1_problem_name,code2,c2_problem_name,similar


In [23]:
def make_train_df(dataset_len, df = dataset) :                              # 트레인 데이터를 만드는 make_train_df 함수. 반복작업으로 하는게 좋지만 귀찮아서 만든 조잡한 사용자 정의 함수이다.
    result = pd.DataFrame()                                                 # 반환할 데이터 프레임
    cnt = 0                                                                 # 반환할 데이터 프레임의 길이.
    
    while cnt < dataset_len :                                               # 중첩 반복문
        df_list = []                                                        # 안쪽의 for문으로 얻은 데이터 프레임을 저장할 df_list
        for j in range(10000) :                                             # 10,000번 반복
            temp_df = train_df.copy()                                       # train_df의 구조를 복사
            sim_num = 0                                                     # 코드 유사성 변수 sim_num
            
            #r = np.random.randint(1, 301, size = 2)                         # problem 300개 중에 2개 뽑기
            
            while True :
                r = np.random.randint(1, 301, size = 2)                     
                
                if r[0] != 209 and r[1] != 209 :
                    break
            
            if r[0] == r[1] :                                               # 만약 뽑은 2개가 같으면 유사성 True
                sim_num = 1
                
            if sim_num == 0 and cnt >= dataset_len / 2 :                    # 뽑은 코드 유사성이 False고, 현재 만든 데이터가 전체의 절반이 넘으면 반복
                continue                                                    # 이유는 True 데이터가 그만큼 안만들어진다. 대략 10,000개 만들면 40~50개.
            
            c1 = np.random.choice(df[df['problem_num'] == r[0]]['code'])    # 각자 해당하면 problem 폴더의 랜덤 코드를 뽑는다.
            c2 = np.random.choice(df[df['problem_num'] == r[1]]['code']) 
            
            temp_df.loc[0] = c1, r[0], c2, r[1], sim_num                    # 위에서 복사한 형식대로 temp_df를 얻고
            
            df_list.append(temp_df)                                         # df_list에 temp_df 추가
            
        cnt += len(df_list)                                                 # cnt에 df_list의 길이 만큼 증가
        print(cnt ,end = ' ')                                               # tqdm보다 이게 보기 편할 거 같아서 print(cnt)
        res = pd.concat(df_list , axis = 0)                                 # df_list를 데이터프레임으로 만들고
        result = pd.concat([result, res], ignore_index = True)              # 최종 반환할 데이터 프레임과 병합
        
    result = result.sample(frac=1, ignore_index=True)[:dataset_len]         # 반환할 데이터 프레임을 랜덤으로 섞고 반환
            
    return result

In [25]:
train_len = 500000                   # 트레인 길이 조절 

train = make_train_df(train_len)

In [ ]:
train['similar'].value_counts()

In [ ]:
train.to_csv('./open_code_train.csv')

In [ ]:
train

# 테스트 파일 전처리

In [26]:
test_df = pd.read_csv('./open/test.csv', encoding='utf-8')

In [27]:
result_df = pd.DataFrame(columns = test_df.columns)
result_df['pair_id'] = test_df['pair_id'].copy()
result_df

,pair_id,code1,code2
0,1,NaN,NaN
1,2,NaN,NaN
2,3,NaN,NaN
3,4,NaN,NaN
4,5,NaN,NaN
...,...,...,...
179695,179696,NaN,NaN
179696,179697,NaN,NaN
179697,179698,NaN,NaN
179698,179699,NaN,NaN


In [28]:
code_list = []

for i in tqdm(range(len(test_df))) :
    code_str = test_df['code1'][i].split('\n')
    py_code = []
    long_annotation = 0
    long_str = 0
    
    for line in code_str :                                     
        line = line.rstrip().lstrip()                           
                    
        # 만약 한 줄 주석으로 시작하면 새로 반복
        if line.startswith('#') :
            continue
        elif line.count('"""') == 0 and line.startswith('\"') and line[1:].endswith('\"') : # "문자열"
            continue  
        elif line.count("'''") == 0 and line.startswith("\'") and line[1:].endswith("\'") :  # '문자열'
            continue
         
        
        # """를 찾았을 때
        if line.count('"""') >= 1 : 
            if line.count('"""') % 2 == 0 and line.startswith('"""') and line[3:].endswith('"""') :
                continue
            elif line.count("'''") % 2 == 0 and line.startswith("'''") and line[3:].endswith("'''") :
                continue  
    
        if line.count('"""') % 2 != 0 and long_annotation == 0 and long_str == 0 : 
            if line.startswith('"""') :         
                long_annotation = 1                          
                continue
            else : 
                long_str = 1
        elif line.count('"""') % 2 != 0 and long_annotation == 1 and long_str == 0 :
            long_annotation = 0
            continue    
        elif line.count('"""') % 2 != 0 and long_annotation == 0 and long_str == 1 :
            long_str = 0
                    
        # '''를 찾았을 때
        if line.count("'''") >= 1 : 
            if line.count('"""') % 2 == 0 and line.startswith('"""') and line[3:].endswith('"""') :
                continue
            elif line.count("'''") % 2 == 0 and line.startswith("'''") and line[3:].endswith("'''") :
                continue  
    
        if line.count("'''") % 2 != 0 and long_annotation == 0 and long_str == 0 : 
            if line.startswith("'''") :         
                long_annotation = 1                          
                continue
            else : 
                long_str = 1
        elif line.count("'''") % 2 != 0 and long_annotation == 1 and long_str == 0 :
            long_annotation = 0
            continue    
        elif line.count("'''") % 2 != 0 and long_annotation == 0 and long_str == 1 :
            long_str = 0
                    
                    
        if long_annotation == 1 :                              # 위에 조건문을 통해서 긴 줄 주석이 없는데 long_annotation이 1이면 
            continue       
            
        if '#' in line :                                        
            if line[:line.index('#')].count('\'') % 2 == 0 and line[line.index('#'):].count('\'') % 2 == 0 :
                line = line[:line.index('#')]                   
                                                                
            elif line[:line.index('#')].count('\"') % 2 == 0 and line[line.index('#'):].count('\"') % 2 == 0 :
                line = line[:line.index('#')]                   
                        
        line = line.replace('\n','')                            
        line = line.replace('    ','\t')                        
                                                                
        py_code.append(line) 

    temp_str = '\n'.join(py_code)                               
    code_list.append(temp_str)
    
result_df['code1'] = pd.DataFrame(code_list)

100%|████████████████████████████████████████████████████████████████████████| 179700/179700 [00:21<00:00, 8447.81it/s]


In [29]:
code_list = []

for i in tqdm(range(len(test_df))) :
    code_str = test_df['code2'][i].split('\n')
    py_code = []
    long_annotation = 0
    long_str = 0
    
    for line in code_str :                                     
        line = line.rstrip().lstrip()                           
                    
        # 만약 한 줄 주석으로 시작하면 새로 반복
        if line.startswith('#') :
            continue
        elif line.count('"""') == 0 and line.startswith('\"') and line[1:].endswith('\"') : # "문자열"
            continue  
        elif line.count("'''") == 0 and line.startswith("\'") and line[1:].endswith("\'") :  # '문자열'
            continue
         
        
        # """를 찾았을 때
        if line.count('"""') >= 1 : 
            if line.count('"""') % 2 == 0 and line.startswith('"""') and line[3:].endswith('"""') :
                continue
            elif line.count("'''") % 2 == 0 and line.startswith("'''") and line[3:].endswith("'''") :
                continue  
    
        if line.count('"""') % 2 != 0 and long_annotation == 0 and long_str == 0 : 
            if line.startswith('"""') :         
                long_annotation = 1                          
                continue
            else : 
                long_str = 1
        elif line.count('"""') % 2 != 0 and long_annotation == 1 and long_str == 0 :
            long_annotation = 0
            continue    
        elif line.count('"""') % 2 != 0 and long_annotation == 0 and long_str == 1 :
            long_str = 0
                    
        # '''를 찾았을 때
        if line.count("'''") >= 1 : 
            if line.count('"""') % 2 == 0 and line.startswith('"""') and line[3:].endswith('"""') :
                continue
            elif line.count("'''") % 2 == 0 and line.startswith("'''") and line[3:].endswith("'''") :
                continue  
    
        if line.count("'''") % 2 != 0 and long_annotation == 0 and long_str == 0 : 
            if line.startswith("'''") :         
                long_annotation = 1                          
                continue
            else : 
                long_str = 1
        elif line.count("'''") % 2 != 0 and long_annotation == 1 and long_str == 0 :
            long_annotation = 0
            continue    
        elif line.count("'''") % 2 != 0 and long_annotation == 0 and long_str == 1 :
            long_str = 0
                    
                    
        if long_annotation == 1 :                              # 위에 조건문을 통해서 긴 줄 주석이 없는데 long_annotation이 1이면 
            continue       
            
        if '#' in line :                                        
            if line[:line.index('#')].count('\'') % 2 == 0 and line[line.index('#'):].count('\'') % 2 == 0 :
                line = line[:line.index('#')]                   
                                                                
            elif line[:line.index('#')].count('\"') % 2 == 0 and line[line.index('#'):].count('\"') % 2 == 0 :
                line = line[:line.index('#')]                   
                        
        line = line.replace('\n','')                            
        line = line.replace('    ','\t')                        
                                                                
        py_code.append(line) 

    temp_str = '\n'.join(py_code)                               
    code_list.append(temp_str)

result_df['code2'] = pd.DataFrame(code_list)

100%|████████████████████████████████████████████████████████████████████████| 179700/179700 [00:20<00:00, 8617.16it/s]


In [35]:
result_df.to_csv('./open_code_test.csv')

# 모델

In [36]:
# problem 파일 경로 Desktop/project/code_similarity/open/code

# problem 폴더 안 예제 코드 확장자명 변경
import os
from tqdm import tqdm
import sys
import tensorflow as tf
import json
import datetime

# .txt 병합
import pandas as pd
import numpy as np
import sklearn

#keras model import
import keras

In [37]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from keras import Input, layers
from tensorflow.keras.layers import Embedding, LSTM, Input, Lambda,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K
from tensorflow.keras.optimizers import Adam
#model save & load
from keras.models import load_model

In [38]:
train_data = pd.read_csv("./open_code_train.csv", index_col = 0, encoding='utf-8')

In [39]:
train_data.drop(['c1_problem_name','c2_problem_name'], axis = 1, inplace= True)
train_data

,code1,code2,similar
0,from collections import deque\nmod = 10**9+7\n...,"from collections import deque\nN, M = [int(x) ...",1
1,"def selection_sort(numbers, n):\ncounter = 0\n...","n,x,t = map(int,(input().split()))\ni = 1\nwhi...",0
2,"n,k,*a = map(int,open(0).read().split())\ndef ...",n = int(input())\nd = {}\nt = 0\nfor i in rang...,0
3,N = str(input())\nprint(chr(ord(N)+1)),k = int(input())\ns = input()\nif len(s)>k:\np...,0
4,"n, a, b = list(map(int, input().split()))\np =...",import sys\ninput = sys.stdin.buffer.readline\...,0
...,...,...,...
499995,s=input()\nans = 0\nfor i in range(len(s)):\nj...,while True:\nhw = [int(x) for x in input().spl...,0
499996,import sys\nn=int(input())\nfor i in range(n+1...,N = int(input())\na = int(N / 1.08) \nb = int(...,1
499997,tmp = [int(e) for e in input().split()]\na = t...,import sys\nfrom functools import reduce\ndef ...,1
499998,n = int(input())\ns = [input() for _ in range(...,"import math\ndef main():\n(N, M, K) = map(int,...",0


In [40]:
print(type(train_data['code1']))
print(type(train_data['code2']))
print(train_data.head())

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
                                               code1  \
0  from collections import deque\nmod = 10**9+7\n...   
1  def selection_sort(numbers, n):\ncounter = 0\n...   
2  n,k,*a = map(int,open(0).read().split())\ndef ...   
3             N = str(input())\nprint(chr(ord(N)+1))   
4  n, a, b = list(map(int, input().split()))\np =...   

                                               code2  similar  
0  from collections import deque\nN, M = [int(x) ...        1  
1  n,x,t = map(int,(input().split()))\ni = 1\nwhi...        0  
2  n = int(input())\nd = {}\nt = 0\nfor i in rang...        0  
3  k = int(input())\ns = input()\nif len(s)>k:\np...        0  
4  import sys\ninput = sys.stdin.buffer.readline\...        0  


In [41]:
test_data = pd.read_csv("./open_code_test.csv", index_col = 0, encoding = 'utf-8')

In [42]:
#test_data = test_data.head()
test_data

,pair_id,code1,code2
0,1,def main():\ns = input()\nif s.count('a') == 1...,"N,K = map(int,input().split())\nA = list(map(i..."
1,2,"N,K,Q = map(int,input().split())\npoints = [0]...","N, K, Q = map(int,input().split())\n\nif K > Q..."
2,3,from itertools import combinations\nn = int(in...,s = input()\nt = input()\nlength_s = len(s)\nl...
3,4,"a,b=map(int,input().split())\n\nans1=a+b\nans2...","a, b, c, d = map(int,input().split())\n\nif a ..."
4,5,S = input()\nK = int(input())\n\nind = -1\nfor...,"H, W = map(int, input().split())\ngrid = []\nf..."
...,...,...,...
179695,179696,N = int(input())\nS = input()\nS_rep = S.repla...,N = int(input())\nS = list(input())\n\nrow = [...
179696,179697,import sys\ndef input(): return sys.stdin.read...,"h,w,a,b = (int(i) for i in input().split())\nf..."
179697,179698,"a = list(map(int, input().split()))\n\na1 = ab...","print('YNEOS'[sorted(input().split())!=['1','4..."
179698,179699,"\n\nH,W,A,B = map(int,input().split())\n\ns = ...","h, w, a, b = map(int, input().split())\nmat = ..."


## 토큰화

In [43]:
tokenizer = Tokenizer() # Tokenizer 객체 생성

In [44]:
tokenizer.fit_on_texts(train_data['code1']) # 단어 인덱스 구축
tokenizer.fit_on_texts(train_data['code2']) # 단어 인덱스 구축

In [45]:
sequences_1 = tokenizer.texts_to_sequences(train_data['code1']) # 문자열을 정수 인덱스의 리스트로 변환한다. 
sequences_2 = tokenizer.texts_to_sequences(train_data['code2'])

In [46]:
train_val = train_data['similar']
train_val.head()

0    1
1    0
2    0
3    0
4    0
Name: similar, dtype: int64

In [47]:
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 고려하여 +1
print(vocab_size)

8930


In [48]:
#가장 긴 sequence의 길이
MAX_SEQUENCE_LENGTH = max([len(seq) for seq in sequences_1 + sequences_2])
ABR__SEQUENCE_LENGTH = round(sum(map(len, sequences_1))/len(sequences_1))
print('코드의 최대 길이 : {}'.format(max([len(seq) for seq in sequences_1 + sequences_2])))
print('코드의 평균길이 : {}'.format(sum(map(len, sequences_1))/len(sequences_1)))
print('코드의 평균길이 : %d'% ABR__SEQUENCE_LENGTH)

코드의 최대 길이 : 128
코드의 평균길이 : 53.031638
코드의 평균길이 : 53


In [49]:
X_train_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
X_train_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
print(X_train_1.shape)
print(X_train_2.shape)
print(type(X_train_2))

(500000, 128)
(500000, 128)
<class 'numpy.ndarray'>


In [50]:
# Model variables
batch_size = 512
epoch = 10
vocab_size = vocab_size
MAX_SEQUENCE_LENGTH = MAX_SEQUENCE_LENGTH
ABR__SEQUENCE_LENGTH = ABR__SEQUENCE_LENGTH
vocab_size = vocab_size
emb_output_dim = 64
DROPOUT_RATIO = 0.25

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
input_tensor1 = layers.Input(shape= (128,), dtype = 'int32', name = 'code1')
input_tensor2 = layers.Input(shape= (128,), dtype = 'int32', name = 'code2')

# Embedding layers
embedding_layer = layers.Embedding(input_dim = vocab_size,
                                   output_dim = emb_output_dim,
                                   input_length = ABR__SEQUENCE_LENGTH,
                                   mask_zero = True,
                                   trainable = False)

# Embedded of the inputs
encoded_1 = embedding_layer(input_tensor1)
encoded_2 = embedding_layer(input_tensor2)

# define lstm for sentence encode
encoded_LSTM1 = layers.LSTM(32, activation='tanh')(encoded_1)
encoded_LSTM2 = layers.LSTM(32, activation='tanh')(encoded_2)

#dropout layer
drop1 = layers.Dropout(DROPOUT_RATIO)(encoded_LSTM1)
drop2 = layers.Dropout(DROPOUT_RATIO)(encoded_LSTM2)

# Calculates the distance as defined by the MaLSTM model
#concats = layers.concatenate([encoded_LSTM1,encoded_LSTM2], axis = -1)

#함수형 API 모델 을 구성할 때 임의 의 Lambda표현식을 사용할 수 있도록 레이어 생성
main_malstm_distance = layers.Lambda(lambda x: exponent_neg_manhattan_distance(x[0], x[1]), output_shape=lambda x: (x[0][0], 1))([drop1,drop2])

#output
#answer = layers.Dense(1, activation='sigmoid')(main_malstm_distance)
# 최종 아웃풋을 어떤것으로 지정할것인가?

#model
malstm = Model([input_tensor1, input_tensor2], outputs = main_malstm_distance)

#model compile 
malstm.compile(loss='mean_squared_logarithmic_error', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

In [51]:
malstm.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 code1 (InputLayer)             [(None, 128)]        0           []                               
                                                                                                  
 code2 (InputLayer)             [(None, 128)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 128, 64)      571520      ['code1[0][0]',                  
                                                                  'code2[0][0]']                  
                                                                                                  
 lstm (LSTM)                    (None, 32)           12416       ['embedding[0][0]']          

In [52]:
# early_stop
early_stopping = EarlyStopping(monitor="val_loss",patience = 10, mode="min")

In [53]:
history = malstm.fit([X_train_1,X_train_2],
                   train_val,
                   verbose=1,
                   callbacks=[early_stopping],
                   validation_split=0.2,
                   epochs=150 ,
                   batch_size = batch_size)   

Epoch 1/150
227/782 [=======>......................] - ETA: 8:31 - loss: 0.1110 - accuracy: 0.5640

KeyboardInterrupt: 

In [ ]:
#model.save('./model_smilarity_MSLE_803')

In [ ]:
# Plot accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# # Plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
#test data 정수
tokenizer.fit_on_texts(test_data['code1']) # 단어 인덱스 구축
tokenizer.fit_on_texts(test_data['code2']) # 단어 인덱스 구축

test_sequences_1 = tokenizer.texts_to_sequences(test_data['code1']) # 문자열을 정수 인덱스의 리스트로 변환한다. 
test_sequences_2 = tokenizer.texts_to_sequences(test_data['code2'])

X_test_1 = pad_sequences(test_sequences_1, maxlen=ABR__SEQUENCE_LENGTH)
X_test_2 = pad_sequences(test_sequences_2, maxlen=ABR__SEQUENCE_LENGTH)

print(X_test_1.shape)
print(X_test_2.shape)
print(type(X_test_1))

In [ ]:
# 모델 추론
preds = malstm.predict(test_data['code1'], test_data['code2'])

In [ ]:
threshold = 0.5
preds = np.where(preds>self.threshold, 1, 0)

In [ ]:
# submission 저장
submission = pd.read_csv('./sample_submission.csv')
submission['similar'] = preds
submission.to_csv('./submission.csv', index=False)